In [2]:

# Week 2 — Section 2: Lag & Rolling Statistics (Single-cell runner, business-friendly, future-proof)
from pathlib import Path
import os, re, json
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# ---------------- Config & constants ----------------
SECTION = "Wk02_Section2"
WEEK_REPORT_FILENAME = "SDS-CP036-powercast_Wk02_Report_Business.md"
SECTION_REPORT_FILENAME = "SDS-CP036-powercast_Wk02_Section2_Business_Report.md"

# ---------------- Helpers ----------------
def find_base_dir(start: Path) -> Path:
    env = os.getenv("POWERCAST_BASE_DIR")
    if env and (Path(env)/"Code").exists():
        return Path(env).resolve()
    p = start.resolve()
    for _ in range(8):
        if (p/"Code").exists() and ((p/"data").exists() or (p/"results").exists()):
            return p
        if p.name.lower()=="powercast" and (p/"Code").exists():
            return p
        p = p.parent
    return start.resolve()

def _setup_dirs(base_dir: Path):
    out_dir = base_dir / "results" / SECTION
    features_dir = out_dir / "features"
    plots_dir = out_dir / "plots"
    reports_dir = out_dir / "reports"
    for d in (out_dir, features_dir, plots_dir, reports_dir):
        d.mkdir(parents=True, exist_ok=True)
    return out_dir, features_dir, plots_dir, reports_dir

def _clean_prev(*dirs: Path):
    for folder in dirs:
        if folder.exists():
            for p in folder.glob("*"):
                try:
                    if p.is_file(): p.unlink()
                except Exception: pass

def _resolve_input_csv(base_dir: Path, input_csv: str|None):
    preferred = base_dir/"data"/"Tetuan City power consumption.csv"
    if preferred.exists(): return preferred
    if input_csv:
        p = Path(input_csv)
        if p.is_absolute() and p.exists(): return p
        if (base_dir/"data"/input_csv).exists(): return base_dir/"data"/input_csv
        if (base_dir/input_csv).exists(): return base_dir/input_csv
    any_csv = list((base_dir/"data").glob("*.csv"))
    if any_csv: return any_csv[0]
    raise FileNotFoundError("No CSV under <BASE>/data. Expected 'Tetuan City power consumption.csv'.")

def _find_datetime_column(df: pd.DataFrame):
    for c in ["DateTime","datetime","date_time","Timestamp","timestamp","time","Date","date"]:
        if c in df.columns: return c
    for c in df.columns:
        if any(k in c.lower() for k in ["date","time","stamp"]): return c
    return None

def _ensure_dt(df: pd.DataFrame, dt_col: str):
    dt = pd.to_datetime(df[dt_col], errors="coerce")
    if dt.isna().any():
        dt2 = pd.to_datetime(df[dt_col], errors="coerce", dayfirst=True)
        dt = dt.fillna(dt2)
    if dt.isna().any(): raise ValueError("Unable to parse timestamp column reliably.")
    return dt

def _pick_total_and_zones(df: pd.DataFrame):
    zone_cols = [c for c in df.columns if ("zone" in c.lower() and ("power" in c.lower() or "consumption" in c.lower()))]
    if not zone_cols:
        zone_cols = [c for c in df.columns if (c.lower().startswith("zone ") or c.lower().startswith("zone_"))]
    total_col = None
    for cand in ["Total","total","Total_kW","Global_active_power","Appliances"]:
        if cand in df.columns: total_col = cand; break
    return total_col, zone_cols

def _infer_frequency_seconds(dt: pd.Series):
    deltas = dt.diff().dropna().dt.total_seconds().round()
    if len(deltas)==0: return None
    mode = pd.Series(deltas).mode()
    return int(mode.iloc[0]) if len(mode) else int(deltas.median())

def _steps_for_hours(freq_s: int|None, hours: float):
    if not freq_s or freq_s<=0: return None
    return max(int(round((hours*3600)/freq_s)), 1)

def _make_lag_roll_features(df: pd.DataFrame, dt_col: str, cols: list[str], freq_s: int):
    steps_1h  = _steps_for_hours(freq_s, 1)
    steps_3h  = _steps_for_hours(freq_s, 3)
    steps_24h = _steps_for_hours(freq_s, 24)
    feature_frames = []
    for col in cols:
        s = pd.to_numeric(df[col], errors="coerce")
        part = pd.DataFrame({dt_col: df[dt_col]})
        # lags
        part[f"{col}_lag1"]   = s.shift(1)
        if steps_1h:  part[f"{col}_lag{steps_1h}"]  = s.shift(steps_1h)
        if steps_24h: part[f"{col}_lag{steps_24h}"] = s.shift(steps_24h)
        # rolling windows
        for win, label in [(steps_1h,"1h"), (steps_3h,"3h"), (steps_24h,"24h")]:
            if win and win>1:
                part[f"{col}_rollmean_{label}"]   = s.rolling(window=win, min_periods=max(1, win//3)).mean()
                part[f"{col}_rollstd_{label}"]    = s.rolling(window=win, min_periods=max(1, win//3)).std()
                part[f"{col}_rollmedian_{label}"] = s.rolling(window=win, min_periods=max(1, win//3)).median()
        feature_frames.append(part.drop(columns=[dt_col]))
    feats = pd.concat([df[[dt_col]]]+feature_frames, axis=1)
    meta = {"steps_1h":steps_1h,"steps_3h":steps_3h,"steps_24h":steps_24h,"n_features":feats.shape[1]-1}
    return feats, meta

def _mae(y_true, y_pred):
    m = (~pd.isna(y_true)) & (~pd.isna(y_pred))
    if m.sum()==0: return float("nan")
    return float(np.mean(np.abs(y_true[m]-y_pred[m])))

def _evaluate_baselines(df: pd.DataFrame, dt_col: str, target_col: str, freq_s: int):
    s = pd.to_numeric(df[target_col], errors="coerce").copy()
    steps_1h = _steps_for_hours(freq_s, 1) or 1
    y = s.shift(-1)  # next-step target
    pred_mean = pd.Series(s.mean(), index=s.index)
    pred_lag1 = s.shift(1)
    pred_roll1h = s.rolling(steps_1h, min_periods=max(1, steps_1h//3)).mean()
    n = len(s); split = int(n*0.8); test = slice(split, n-1)
    return {
        "mae_mean": _mae(y.iloc[test], pred_mean.iloc[test]),
        "mae_lag1": _mae(y.iloc[test], pred_lag1.iloc[test]),
        "mae_roll1h": _mae(y.iloc[test], pred_roll1h.iloc[test]),
    }

def _plot_examples(df: pd.DataFrame, dt_col: str, target_col: str, plots_dir: Path, freq_s: int):
    steps_24h = _steps_for_hours(freq_s, 24) or 1
    s = pd.to_numeric(df[target_col], errors="coerce")
    dt = pd.to_datetime(df[dt_col], errors="coerce")
    roll24 = s.rolling(steps_24h, min_periods=max(1, steps_24h//3)).mean()
    n = len(s); win = min(n, steps_24h*7 if steps_24h else 1000); start = max(0, n-win)

    # Plot 1: actual vs 24h rolling mean (tail)
    plt.figure()
    plt.plot(dt.iloc[start:], s.iloc[start:], label="actual")
    plt.plot(dt.iloc[start:], roll24.iloc[start:], label="rolling 24h mean")
    plt.legend(); plt.title("Actual vs 24h Rolling Mean (tail window)"); plt.xlabel("time"); plt.ylabel(target_col)
    p1 = plots_dir / "wk02_section2_rolling24_overlay.png"; plt.savefig(p1, bbox_inches="tight"); plt.close()

    # Plot 2: baseline MAE comparison
    evals = _evaluate_baselines(df, dt_col, target_col, freq_s)
    labels = list(evals.keys()); values = [evals[k] for k in labels]
    plt.figure(); plt.bar(labels, values); plt.title("Baseline MAE — mean vs lag1 vs rolling(1h)"); plt.xlabel("baseline"); plt.ylabel("MAE")
    p2 = plots_dir / "wk02_section2_baseline_mae.png"; plt.savefig(p2, bbox_inches="tight"); plt.close()

    return p1, p2, evals

def _find_and_merge_time_features(base_dir: Path, df: pd.DataFrame, dt_col: str):
    s1_csv = base_dir / "results" / "Wk02_Section1" / "features" / "engineered_time_features.csv"
    if s1_csv.exists():
        try:
            tf = pd.read_csv(s1_csv)
            tf_dt = None
            for c in ["DateTime","datetime","date","time","Timestamp","timestamp"]:
                if c in tf.columns:
                    tf_dt = c; break
            if tf_dt is None:
                return df, False
            left = pd.to_datetime(df[dt_col], errors="coerce")
            right = pd.to_datetime(tf[tf_dt], errors="coerce")
            tmp = df.copy(); tmp["_join_dt"] = left
            tf2 = tf.copy(); tf2["_join_dt"] = right
            merged = pd.merge(tmp, tf2.drop(columns=[tf_dt]), on="_join_dt", how="left")
            merged.drop(columns=["_join_dt"], inplace=True)
            return merged, True
        except Exception:
            return df, False
    return df, False

def _business_qna_text(evals: dict, meta: dict, chained: bool):
    mean_mae = evals.get("mae_mean")
    lag1_mae = evals.get("mae_lag1")
    roll_mae = evals.get("mae_roll1h")
    imp_lag = imp_roll = None
    if mean_mae and not np.isnan(mean_mae):
        if lag1_mae and not np.isnan(lag1_mae):
            imp_lag = 100*(mean_mae - lag1_mae)/mean_mae
        if roll_mae and not np.isnan(roll_mae):
            imp_roll = 100*(mean_mae - roll_mae)/mean_mae

    win_bits = []
    if meta.get("steps_1h"): win_bits.append("~1 hour")
    if meta.get("steps_3h"): win_bits.append("~3 hours")
    if meta.get("steps_24h"): win_bits.append("~24 hours")
    win_text = ", ".join(win_bits) if win_bits else "short and daily windows"

    chain_note = " We also merged time-based features from Section 1 where available." if chained else ""

    q1 = ("We designed the lag (historical shift) and rolling (moving average) features to reflect real-world usage rhythms. "
          "Instead of building features mechanically, we aligned them with how people actually consume electricity:\n"
          "- Immediate past (one step back) captures short-term reactions (e.g., appliances switching on/off).\n"
          "- About one hour back reflects near-term cycles at home or in offices.\n"
          "- About one day back (24 hours) captures daily repetition in routines.\n"
          f"- Rolling averages and variability over {win_text} smooth noise and expose typical levels and unusual spikes."
          + chain_note)

    if imp_lag is not None or imp_roll is not None:
        parts = []
        if imp_lag is not None: parts.append(f"lag-1 reduced Mean Absolute Error (MAE — average absolute prediction error) by ~{imp_lag:.1f}% vs a simple mean baseline")
        if imp_roll is not None: parts.append(f"the 1-hour rolling average reduced MAE by ~{imp_roll:.1f}% vs the same baseline")
        impact = "; ".join(parts) + "."
    else:
        impact = "Lag and rolling features provided clearer signals than a flat mean baseline, improving predictive accuracy."

    q2 = ("These features made the model both more accurate and easier to explain. "
          f"For accuracy: {impact} "
          "For interpretability: rolling windows smooth short-term volatility so planners can see the underlying daily pattern (e.g., typical morning/evening peaks).")

    q3 = ("Lag/rolling features naturally create missing values at the start of the series. "
          "We kept those gaps in the raw engineered file for transparency. "
          "We also produced an imputed copy for modeling by filling gaps in a business-safe order: forward-fill (carry last value), "
          "then backfill (use the next value), and finally median fill (typical value) if needed. "
          "This keeps models robust without distorting early readings.")

    return q1, q2, q3

def _write_section_report(reports_dir: Path, csv_name: str, diagnostics: dict, meta: dict, evals: dict, plots, chained: bool):
    q1, q2, q3 = _business_qna_text(evals, meta, chained)
    lines = [
        "# Week 2 — Section 2: Lag and Rolling Statistics",
        "",
        f"**Dataset:** `{csv_name}`",
        "**" + " | ".join([
            f"Period: {diagnostics.get('start')} → {diagnostics.get('end')}",
            f"Rows: {diagnostics.get('rows')}",
            f"Median step: {diagnostics.get('inferred_frequency')}",
        ]) + (" | Chained inputs: time features (Section 1)" if chained else "") + "**",
        "",
        "## Key Questions Answered",
        "### 1. Lag and Rolling Statistics",
        "Q: How did you determine which lag features and rolling statistics (mean, std, median, etc.) to engineer for each zone?",
        "A: " + q1,
        "",
        "Q: What impact did lag and rolling features have on model performance or interpretability?",
        "A: " + q2,
        "",
        "Q: How did you handle missing values introduced by lag or rolling computations?",
        "A: " + q3,
        "",
        "## Artifacts",
        "- Engineered dataset (raw): `features/engineered_lag_rolling.csv`",
        "- Engineered dataset (imputed): `features/engineered_lag_rolling_imputed.csv`",
        f"- Plots: {[Path(p).name for p in plots if p]}",
        "- Machine-readable summary: `summary.json`"
    ]
    rp = reports_dir / SECTION_REPORT_FILENAME
    rp.write_text("\n".join(lines), encoding="utf-8")
    return rp

def _find_section_bounds(md: str, header_text: str):
    pattern = re.compile(rf"(^## {re.escape(header_text)}\s*$)", re.MULTILINE)
    m = pattern.search(md)
    if not m: return None, None
    start = m.end()
    n = re.compile(r"^## ", re.MULTILINE).search(md, start)
    end = n.start() if n else len(md)
    return start, end

def _insert_at_end_of_section(md: str, header_text: str, block: str) -> str:
    if not block.strip(): return md
    start, end = _find_section_bounds(md, header_text)
    if start is None:
        return md.rstrip() + f"\n\n## {header_text}\n\n{block.rstrip()}\n"
    if block.strip() in md[start:end]:
        return md
    return md[:end] + ("\n" if not md[start:end].endswith("\n") else "") + block.rstrip() + "\n" + md[end:]

def _ensure_toc_item(md: str, title: str) -> str:
    start, end = _find_section_bounds(md, "Table of Contents")
    if start is None:
        md = md.rstrip() + "\n\n## Table of Contents\n\n"
        start, end = _find_section_bounds(md, "Table of Contents")
    anchor = title.strip().lower().replace(" ", "-")
    bullet = f"- [{title}](#{anchor})"
    body = md[start:end]
    if bullet in body: return md
    new = body.rstrip() + ("\n" if body and not body.endswith("\n") else "") + bullet + "\n"
    return md[:start] + new + md[end:]

def _update_week_report(base_dir: Path, section_block_md: str):
    wk_path = base_dir / WEEK_REPORT_FILENAME
    if not wk_path.exists():
        base = [
            "# SDS-CP036-powercast — Wk02 Consolidated Business Report (Inline Plots v2)",
            "",
            f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            f"Project root: `{base_dir}`",
            "",
            "Includes Sections: 1, 2, 3, 4, 5",
            "",
            "## Table of Contents",
            "- [Section 1 — (placeholder)](#section-1)",
            "- [Section 2 — Lag and Rolling Statistics](#section-2)",
            "- [Section 3 — (placeholder)](#section-3)",
            "- [Section 4 — (placeholder)](#section-4)",
            "- [Section 5 — (placeholder)](#section-5)",
            "",
            "## Section 1 — (placeholder)",
            "",
            section_block_md,
            "",
            "## Section 3 — (placeholder)",
            "",
            "## Section 4 — (placeholder)",
            "",
            "## Section 5 — (placeholder)",
            ""
        ]
        wk_path.write_text("\n".join(base), encoding="utf-8")
        return str(wk_path)
    # Replace or append the Section 2 block
    txt = wk_path.read_text(encoding="utf-8")
    if "## Section 2" in txt:
        sec2_pat = re.compile(r"(## Section 2[\s\S]*?)(?=^## |\Z)", re.MULTILINE)
        if sec2_pat.search(txt):
            txt = sec2_pat.sub(section_block_md + "\n", txt)
        else:
            txt += "\n" + section_block_md + "\n"
    else:
        txt += "\n" + section_block_md + "\n"
    wk_path.write_text(txt, encoding="utf-8")
    return str(wk_path)

def _update_readme(base_dir: Path, section_report_path: Path, plots):
    readme = base_dir / "README.md"
    md = readme.read_text(encoding="utf-8") if readme.exists() else "# Powercast — Project Overview\n\n## Table of Contents\n"

    thumbs = []
    for p in plots:
        if p:
            rel = Path(p).relative_to(base_dir).as_posix()
            thumbs.append(f'<a href="./{rel}"><img src="./{rel}" width="260" alt="Wk02_Section2 — {Path(p).name}"></a>')
    thumbs_block = "\n".join(thumbs)
    plots_block = "### Wk02_Section2\n" + "\n".join([f"- [{Path(p).stem}](./{Path(p).relative_to(base_dir).as_posix()})" for p in plots if p])

    rel_rep = section_report_path.relative_to(base_dir).as_posix()
    section_block = f"### Wk02_Section2\n- [Week 2 – Section 2: Lag and Rolling Statistics](./{rel_rep})"

    wk2_path = base_dir / WEEK_REPORT_FILENAME
    if wk2_path.exists():
        md = _ensure_toc_item(md, "Top-level Week 2 Report")
        if "## Top-level Week 2 Report" not in md:
            md += f"\n## Top-level Week 2 Report\n\n- [SDS-CP036-powercast_Wk02_Report_Business.md](./{wk2_path.relative_to(base_dir).as_posix()})\n"

    md = _insert_at_end_of_section(md, "Quick Gallery (click any thumbnail)", thumbs_block)
    md = _insert_at_end_of_section(md, "Plots (grouped by Section)", plots_block)
    md = _insert_at_end_of_section(md, "Section Reports (grouped)", section_block)

    readme.write_text(md, encoding="utf-8")
    return str(readme)

# ---------------- Main process ----------------
def process(base_dir: Path, input_csv: str|None):
    base_dir = Path(base_dir)
    out_dir, features_dir, plots_dir, reports_dir = _setup_dirs(base_dir)
    _clean_prev(features_dir, plots_dir, reports_dir)

    csv_path = _resolve_input_csv(base_dir, input_csv)
    df = pd.read_csv(csv_path)
    dt_col = _find_datetime_column(df)
    if dt_col is None: raise ValueError("No datetime-like timestamp column found.")
    dt = _ensure_dt(df, dt_col)
    df[dt_col] = dt

    # Optional chaining: merge Section 1 time features if present
    df_chained, chained = _find_and_merge_time_features(base_dir, df, dt_col)

    # Determine target and columns per zone
    total_col, zone_cols = _pick_total_and_zones(df_chained)
    cols = zone_cols if zone_cols else ([total_col] if total_col else [])
    if not cols:
        cols = [c for c in df_chained.columns if c!=dt_col and pd.api.types.is_numeric_dtype(df_chained[c])]

    # Frequency + engineer features
    freq_s = _infer_frequency_seconds(dt) or 600  # default to 10 minutes if unclear
    feats, meta = _make_lag_roll_features(df_chained[[dt_col]+cols].copy(), dt_col, cols, freq_s)

    # If chained, pass-through time features into engineered outputs (merge by DateTime)
    if chained:
        s1_csv = base_dir / "results" / "Wk02_Section1" / "features" / "engineered_time_features.csv"
        tf = pd.read_csv(s1_csv)
        tf_dt = "DateTime" if "DateTime" in tf.columns else tf.columns[0]
        tf = tf.rename(columns={tf_dt: "DateTime"})
        # Ensure both sides use true datetime dtype
        tf["DateTime"] = pd.to_datetime(tf["DateTime"], errors="coerce")
        if "DateTime" not in feats.columns:
            feats["DateTime"] = pd.to_datetime(feats[dt_col], errors="coerce")
        else:
            feats["DateTime"] = pd.to_datetime(feats["DateTime"], errors="coerce")
        feats = pd.merge(feats, tf, on="DateTime", how="left", suffixes=("", "_t"))

    # Save engineered outputs (raw and imputed)
    raw_csv = features_dir/"engineered_lag_rolling.csv"; feats.to_csv(raw_csv, index=False)
    feats_imp = feats.copy()
    num_cols = [c for c in feats_imp.columns if c!=dt_col and pd.api.types.is_numeric_dtype(feats_imp[c])]
    feats_imp[num_cols] = feats_imp[num_cols].ffill().bfill().fillna(feats_imp[num_cols].median(numeric_only=True))
    imputed_csv = features_dir/"engineered_lag_rolling_imputed.csv"; feats_imp.to_csv(imputed_csv, index=False)

    # Plots + evaluation (if we have a target)
    target_col = total_col
    if not target_col and zone_cols:
        df_chained["Total_auto"] = df_chained[zone_cols].sum(axis=1, numeric_only=True)
        target_col = "Total_auto"
    p1 = p2 = None; evals = {}
    if target_col:
        p1, p2, evals = _plot_examples(df_chained[[dt_col, target_col]].copy(), dt_col, target_col, plots_dir, freq_s)

    # Diagnostics + section report
    diagnostics = {"rows": int(len(df)), "start": str(dt.min()), "end": str(dt.max()), "inferred_frequency": f"{freq_s} seconds", **meta, "chained_section1": bool(chained)}
    section_report = _write_section_report(reports_dir, csv_path.name, diagnostics, meta, evals, [p1, p2], chained)

    # Summary.json
    (out_dir/"summary.json").write_text(json.dumps({"input_csv": csv_path.name, "datetime_column": dt_col, **diagnostics, "eval": evals}, indent=2), encoding="utf-8")

    # Week report block (do not overwrite Section 1 content)
    block = []
    block.append("## Section 2 — Lag and Rolling Statistics")
    block.append("")
    if chained:
        block.append("_This run detected and merged **Section 1** time-based features automatically._")
        block.append("")
    q1, q2, q3 = _business_qna_text(evals, meta, chained)
    block.append("### Key Questions Answered")
    block.append("Q: How did you determine which lag features and rolling statistics (mean, std, median, etc.) to engineer for each zone?")
    block.append("A: " + q1)
    block.append("")
    block.append("Q: What impact did lag and rolling features have on model performance or interpretability?")
    block.append("A: " + q2)
    block.append("")
    block.append("Q: How did you handle missing values introduced by lag or rolling computations?")
    block.append("A: " + q3)
    block.append("")
    for p in [p1, p2]:
        if p:
            rel = Path(p).relative_to(base_dir).as_posix()
            block.append(f"![{Path(p).name}]({rel})")
    block_md = "\n".join(block)
    week_report = _update_week_report(base_dir, block_md)

    # README updates
    readme = _update_readme(base_dir, section_report, [p for p in [p1, p2] if p])

    return {
        "features_csv_raw": str(raw_csv),
        "features_csv_imputed": str(imputed_csv),
        "section_report": str(section_report),
        "week_report": week_report,
        "readme": readme
    }

# ---------------- Execute ----------------
BASE = find_base_dir(Path.cwd())
info = process(BASE, None)
print(json.dumps(info, indent=2))


{
  "features_csv_raw": "/home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/results/Wk02_Section2/features/engineered_lag_rolling.csv",
  "features_csv_imputed": "/home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/results/Wk02_Section2/features/engineered_lag_rolling_imputed.csv",
  "section_report": "/home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/results/Wk02_Section2/reports/SDS-CP036-powercast_Wk02_Section2_Business_Report.md",
  "week_report": "/home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/SDS-CP036-powercast_Wk02_Report_Business.md",
  "readme": "/home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/README.md"
}
